In [1]:
%matplotlib notebook
# imports
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud
from nuscenes.utils.geometry_utils import transform_matrix
from pyquaternion import Quaternion

from IPython.display import HTML

matplotlib.use('TkAgg')

In [2]:

# basic config and variables
_VERSION = 'v1.0-mini'
_DATAROOT = '../data'

nusc = NuScenes(version=_VERSION, dataroot=_DATAROOT, verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.250 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
def get_pcd_data(nusc_filepath: str):
    radar_point_cloud = RadarPointCloud.from_file(nusc_filepath)
    points =  radar_point_cloud.points
    x = points[0]
    y = points[1]
    vx_comp = points[8]
    vy_comp = points[9]

    return {
        'file': nusc_filepath,
        'x': x,
        'y': y,
        'vx_comp': vx_comp,
        'vy_comp': vy_comp,
        'v_comp': (vx_comp ** 2 + vy_comp ** 2) ** 0.5,
        'radar_point_cloud': radar_point_cloud
    }

def extract_channel_from_file(channel: str):
    filename = nusc.get('sample_data', channel)['filename']
    filename = f'{_DATAROOT}/{filename}'
    return get_pcd_data(filename)


def extract_samples_from_scene(scene: dict):
    """extract all the samples related to the given scene."""
    first_sample_token = scene['first_sample_token']
    last_sample_token = scene['last_sample_token']
    samples = list()

    fst = nusc.get('sample', first_sample_token)
    next_token = fst['next']
    while True:
        current = nusc.get('sample', next_token)
        samples.append(current)
        next_token = current['next']
        if next_token == last_sample_token:
            return samples


def convert_binary_data_to_coordinates_and_velocity(sample: dict):
    data = sample['data']
    return {
     'RADAR_FRONT' : extract_channel_from_file(data['RADAR_FRONT']),
     'RADAR_FRONT_LEFT' : extract_channel_from_file(data['RADAR_FRONT_LEFT']),
     'RADAR_FRONT_RIGHT' : extract_channel_from_file(data['RADAR_FRONT_RIGHT']),
     'RADAR_BACK_LEFT' : extract_channel_from_file(data['RADAR_BACK_LEFT']),
     'RADAR_BACK_RIGHT' : extract_channel_from_file(data['RADAR_BACK_RIGHT']),
     'data': data
    }

sc = nusc.scene[0]
samples_from_scene = extract_samples_from_scene(sc)
scene_in_sample_data = [convert_binary_data_to_coordinates_and_velocity(sc) for sc in samples_from_scene]


In [4]:
def new_method():
    """makes a biggo matrix containing all infos about 2 a)."""
    channels = [
        'RADAR_FRONT',
        'RADAR_FRONT_LEFT',
        'RADAR_FRONT_RIGHT',
        'RADAR_BACK_LEFT',
        'RADAR_BACK_RIGHT',
    ]
    samples = scene_in_sample_data
    scene_points = list()
    for sample in samples:
        x, y, z, vx_comp, vy_comp, pointclouds = list(), list(), list() ,list() ,list(), list()
        ego_pose_coords = []
        for channel in channels:
            pc = sample[channel]['radar_point_cloud']
            radar_token = sample['data'][channel]
            current_radar = nusc.get('sample_data', radar_token)
            ego_pose = nusc.get('ego_pose', current_radar['ego_pose_token'])
            calibrated_sensor = nusc.get('calibrated_sensor', current_radar['calibrated_sensor_token'])
            sensor_to_car = transform_matrix(calibrated_sensor['translation'], Quaternion(calibrated_sensor['rotation'], inverse=False))
            car_to_world = transform_matrix(ego_pose['translation'], Quaternion(ego_pose['rotation'], inverse=False))

            sensor_to_world = car_to_world @ sensor_to_car

            pc.transform(sensor_to_world)

            pointclouds.append(pc)
            
            ego_pose_coords = ego_pose['translation']

            # combine radar
            
            for i in range(pc.points.shape[1]):
                x.append(pc.points[0][i])
                y.append(pc.points[1][i])
                z.append(pc.points[2][i]) # redundant?
                vx_comp.append(pc.points[7][i])
                vy_comp.append(pc.points[8][i])
        scene_points.append([
            np.asarray(x),
            np.asarray(y),
            np.asarray(z),
            np.asarray(vx_comp),
            np.asarray(vy_comp),
            np.asarray(pointclouds),
            np.asarray(ego_pose_coords)
        ])

    return np.asarray(scene_points, dtype=object)


result = new_method()

In [65]:
plt.style.use('dark_background')

ys = list()
xs = list()
ego_x = list()
ego_y = list()

result_iterator = iter(result)

def set_color(x, y) -> [str]:
    t = [v[0] < 0 or v[1] < 0 for v in zip(x,y)]
    res = list()
    for b in t:
        if b:
            res.append('aquamarine')
        else:
            res.append('magenta')
    return res


def get_point_coords_from_v_comp(vx_comp, vy_comp, ego_x, ego_y):
    distance_x = vx_comp * 2 # 2 Hz
    distance_y = vy_comp * 2 # 2 Hz
    return (distance_x + ego_x, distance_y + ego_y)
    

def update(i):
    try:
        row = next(result_iterator)
    except StopIteration:
        return
    x, y = row[0], row[1]
    vx, vy = row[3], row[4]
    radar_ego_x = row[6][0]
    radar_ego_y = row[6][1]
    
    points_x = get_point_coords_from_v_comp(vx,vy,x, y)[0]
    points_y = get_point_coords_from_v_comp(vx,vy,x, y)[1]
    points = zip(points_x, points_y)
    print(list(points)[0], (radar_ego_x,  radar_ego_y))

    plt.cla()
    plt.ylim(900, 1300)
    plt.xlim(300, 550)


    plt.scatter(x, y, s=1, color=set_color(vx, vy))
    plt.scatter(radar_ego_x, radar_ego_y, s=50, color='red')
    plt.plot()


ani = animation.FuncAnimation(plt.gcf(), update, frames=len(result), interval=100)
# plt.show()

HTML(ani.to_html5_video())

(411.0814577390558, 1168.7725940779362) (409.7480833069988, 1176.6904246490058)
(408.0097606508839, 1163.5255990835144) (408.0109097208468, 1171.8897855892765)
(407.98311250531526, 1161.6774928547977) (406.7970472415747, 1168.4160026103984)
(394.4929578161434, 1157.7642811949024) (405.4416839411389, 1164.4493610045051)
(392.45380893300796, 1155.056760589457) (404.35274972398, 1161.2720631277261)
(403.7194028273652, 1139.1982749462763) (403.153956334162, 1157.6388726785221)
(403.2683460645797, 1138.262592977614) (401.8446961665583, 1153.6031748017879)
(404.05216601156434, 1139.2337666473925) (400.7718761781814, 1150.1816384000194)
(403.54689188052845, 1138.852261402342) (399.89359234281727, 1147.3411782218257)
(398.5710815596661, 1127.9788957564344) (398.95916621815115, 1144.1869471977031)
(397.9645908711955, 1127.8633211222639) (398.2070611334777, 1141.5172518051722)
(397.9263805509064, 1128.3538171464606) (397.4663374202455, 1138.5529194210922)
(397.93617252697055, 1128.4324835634502)

In [41]:
doppl = [(r[3],r[4]) for r in result]


# doppl[0]
#
# for d in doppl:
#     r = zip(d[0], d[1])
#
#     for v in r:
#         print(v[0] < 0 or v[1] < 0, v)


{'xkcd:cloudy blue': '#acc2d9',
 'xkcd:dark pastel green': '#56ae57',
 'xkcd:dust': '#b2996e',
 'xkcd:electric lime': '#a8ff04',
 'xkcd:fresh green': '#69d84f',
 'xkcd:light eggplant': '#894585',
 'xkcd:nasty green': '#70b23f',
 'xkcd:really light blue': '#d4ffff',
 'xkcd:tea': '#65ab7c',
 'xkcd:warm purple': '#952e8f',
 'xkcd:yellowish tan': '#fcfc81',
 'xkcd:cement': '#a5a391',
 'xkcd:dark grass green': '#388004',
 'xkcd:dusty teal': '#4c9085',
 'xkcd:grey teal': '#5e9b8a',
 'xkcd:macaroni and cheese': '#efb435',
 'xkcd:pinkish tan': '#d99b82',
 'xkcd:spruce': '#0a5f38',
 'xkcd:strong blue': '#0c06f7',
 'xkcd:toxic green': '#61de2a',
 'xkcd:windows blue': '#3778bf',
 'xkcd:blue blue': '#2242c7',
 'xkcd:blue with a hint of purple': '#533cc6',
 'xkcd:booger': '#9bb53c',
 'xkcd:bright sea green': '#05ffa6',
 'xkcd:dark green blue': '#1f6357',
 'xkcd:deep turquoise': '#017374',
 'xkcd:green teal': '#0cb577',
 'xkcd:strong pink': '#ff0789',
 'xkcd:bland': '#afa88b',
 'xkcd:deep aqua': '#0